In [1]:
!pip --version

pip 24.0 from /usr/local/python/3.10.13/lib/python3.10/site-packages/pip (python 3.10)


In [2]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 12.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

In [12]:
es_client = Elasticsearch('http://localhost:9200')

In [21]:
if es_client.ping():
    print("Connected to Elasticsearch!")
else:
    print("Could not connect to Elasticsearch.")


Connected to Elasticsearch!


In [57]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [58]:
!curl localhost:9200

{
  "name" : "6664268ed0ce",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "8dlEEfZhSayLrAX8PLvzDw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [14]:
health = es_client.cluster.health()
print(health)


{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [38]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/VX6xnjsyRzGF6xzGRF5ygQ] already exists')

In [52]:
es_client.indices.refresh(index="course-questions")


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [54]:
es_client.search(
    index="course-questions",
    body={
        "query": {
            "match_all": {}
        }
    }
)


ObjectApiResponse({'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1896, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'course-questions', '_id': 'wO-BSpABeb47UQi6V-1m', '_score': 1.0, '_ignored': ['text.keyword'], '_source': {'text': 'In this section of the course, the 5432 port of pgsql is mapped to your computer’s 5432 port. Which means you can access the postgres database via pgcli directly from your computer.\nSo No, you don’t need to run it inside another container. Your local system will do.', 'section': 'Module 1: Docker and Terraform', 'question': 'PGCLI - INKhould we run pgcli inside another docker container?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'we-BSpABeb47UQi6V-2B', '_score': 1.0, '_ignored': ['text.keyword'], '_source': {'text': 'FATAL:  password authentication failed for user "root"\nobservations: Below in bold do not forget the folder t

In [19]:
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:24<00:00, 38.32it/s]


In [39]:
query = 'how do I run kafka?'


In [48]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}


In [49]:
response = es_client.search(index=index_name, body=search_query)

In [55]:
es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [51]:
response['hits']

{'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}

In [56]:
index_name

'course-questions'

## Home Work Solution:

In [60]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()



In [63]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [64]:
len(documents)

948

In [65]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}